# Task 5 - Viral Tweets Prediction Challenge

Débora Mayumi Rissato - 5288223

Douglas Decicino de Andrade - 10883512

Paulino Ribeiro Villas Boas - 2950178

Renan Silva Chun - 10691817

Renan de Oliveira da Cruz - 10801090

Notebook com todas as saídas no link: https://github.com/nan-oliveira/ML/blob/main/T05%20-%20Twitter/Task5_final_tweets.ipynb

Notebook contendo a análise dos dados da competição "Viral Tweets Prediction Challenge" (https://bitgrit.net/competition/12). O objetivo desta competição é desenvolver um modelo de aprendizado de máquina para prever o nível de "viralidade" de cada tweet com base em atributos como conteúdo do tweet, mídia anexada ao tweet e data/hora de publicação.

Abaixo temos o import de algumas bibliotecas que serão utilizadas no decorrer do código.

In [ ]:
from sklearn.decomposition import PCA
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import (RepeatedStratifiedKFold, cross_validate,
                                     train_test_split, GridSearchCV,cross_val_score)
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
import random
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [ ]:
#Dados treino
train_tweets = pd.read_csv("Dataset/Tweets/train_tweets.csv")
train_tweets_vectorized_media = pd.read_csv("Dataset/Tweets/train_tweets_vectorized_media.csv")
train_tweets_vectorized_text = pd.read_csv("Dataset/Tweets/train_tweets_vectorized_text.csv")

#Dados teste
test_tweets = pd.read_csv("Dataset/Tweets/test_tweets.csv")
test_tweets_vectorized_media = pd.read_csv("Dataset/Tweets/test_tweets_vectorized_media.csv")
test_tweets_vectorized_text = pd.read_csv("Dataset/Tweets/test_tweets_vectorized_text.csv")

In [ ]:
user_vectorized_descriptions = pd.read_csv("Dataset/Users/user_vectorized_descriptions.csv")
user_vectorized_profile_images = pd.read_csv("Dataset/Users/user_vectorized_profile_images.csv")
users = pd.read_csv("Dataset/Users/users.csv")

In [ ]:
print(train_tweets_vectorized_media.shape)
print(train_tweets_vectorized_text.shape)
print(train_tweets.shape)

Abaixo verificamos se os dados contêm valores ausentes.

In [ ]:
train_tweets.isnull().mean()

In [ ]:
sum(train_tweets_vectorized_media.isnull().mean())

In [ ]:
sum(train_tweets_vectorized_text.isnull().mean())

In [ ]:
sum(user_vectorized_descriptions.isnull().mean())

In [ ]:
sum(user_vectorized_profile_images.isnull().mean())

In [ ]:
sum(users.isnull().mean())

Temos que apenas a variável 'tweet_topic_ids' possui valores ausentes, porém iremos descartá-la no decorrer do projeto.

Abaixo temos algumas informações da base de dados.

In [ ]:
train_tweets.info()

In [ ]:
train_tweets_vectorized_media.info()

In [ ]:
train_tweets_vectorized_text.info()

In [ ]:
user_vectorized_descriptions.info()

In [ ]:
user_vectorized_profile_images.info()

In [ ]:
users.info()

## 1 Análise descritiva

Nesta seção apresenta-se uma análise descritiva dos dados.

### 1.1 Tweets

Aqui, faremos uma análise descritiva dos dados relacionados aos Tweets. 
Os dados referentes aos tweets estão separados em 3 CSVs, sendo eles:

* 1 - Dados dos tweets;
* 2 - Dados referentes as mídias presentes nos tweets (essas informações são dados em forma vetorizada);
* 3 - Dados referentes ao texto presente nos tweets (essas informações são dados em forma vetorizada).

Assim, vejamos a proporção da variável resposta, isto é, como ela está distribuída.

In [ ]:
df = train_tweets
df.head()
x,y = 'index', 'virality'

df1 = df[y].value_counts(normalize=True)
df1 = df1.mul(100)
df1 = df1.rename('percent').reset_index()

g = sns.catplot(x=x,y='percent',kind='bar',data=df1)
g.ax.set_ylim(0,50)

for p in g.ax.patches:
    txt = str(p.get_height().round(2)) + '%'
    txt_x = p.get_x() 
    txt_y = p.get_height()
    g.ax.text(txt_x,txt_y,txt)
    
plt.title("Distribuição da resposta - virality")
plt.xlabel("virality")
plt.ylabel("Proporção")
plt.show()

Podemos ver que os dados apresentam um desbalanceamento considerável em relação à variável resposta, sendo tweets mais virais menos frequêntes que tweets menos virais na base.

Ainda, apresentamos algumas medidas destritivas do dataset train_tweets.

In [ ]:
train_tweets.describe()

Em seguida, fazemos alguns boxplots para verificação de outliers e para a averiguação de como estão destribuídos os atritudos numéricos do data.frame train_tweets.

In [ ]:
for col in ['tweet_created_at_year', 'tweet_created_at_hour', 'tweet_hashtag_count',
            'tweet_url_count',  'tweet_mention_count']:
    sns.boxplot(x = train_tweets[col])
    plt.show()

Com os box-plots, não encontramos nenhum valor outlier que esteja fora do normal de acordo com os atributos.

Abaixo apresentamos uma matriz de correlação dos dados dos tweets.

In [ ]:
plt.figure(figsize=(13, 9))
       
corrMatrix = train_tweets.corr()
sns.heatmap(corrMatrix, annot=True)
plt.show()

Não encontramos nenhuma correlação muito forte entre as features.

### 1.1 Users

Agora, será desenvolvida uma análise para os dados relacionados aos usuários.

Vejamos abaixo as dimensões dos datasets relacionados aos Usuários.

In [ ]:
user_vectorized_profile_images.shape

In [ ]:
user_vectorized_descriptions.shape

In [ ]:
users.shape

In [ ]:
users

Segue abaixo a estimativa kernel das distribuições dos atributos relacionados aos usuários.

In [ ]:
for  col in ['user_like_count', 'user_followers_count',
             'user_following_count', 'user_listed_on_count',
             'user_tweet_count', 'user_created_at_year',
             'user_created_at_month']:
    sns.kdeplot(data=users, x = col, fill="stack")
    plt.show()

Vemos que nenhum atributo possui uma distribuição muito singular. A maioria deles são distribuições de forma assimétrica à direita.


Abaixo temos a matriz de correlação dos atributos relacionados aos usuários.

In [ ]:
plt.figure(figsize=(13, 9))

corrMatrix = users.corr()
sns.heatmap(corrMatrix, annot=True)
plt.show()

### 1.2 Juntando os dados

#### 1.2.1 Funções

A função MergeMedias foi criada para fazer a média dos atributos das midias de cada tweet_id, pois não foi possível concatenar todas as midias de cada tweet. Se fizéssemos isso, haveria muitos atributos esparsos na matriz de atributos de tweets. Assim, decidimos trabalhar com a média dos tweets. Entretanto, não notamos melhoras nos classificadores e até fizemos um teste para verificar se apenas os atribubos médios das midias seriam capazes de idenficar tweets virais. Para a nossa surpresa, todos os classificadores testados predizeram apenas a viralidade "1". Logo, concluímos que os atributos das midias não acrescentam informação útil para a classificação da viralidade dos tweets.
**Com base nesta análise, decidimos não trabalhar com os atributos das medias**.

In [ ]:
def MergeMedias(media,tweets):
    tweet_id = tweets.tweet_id.values
    
    names = media.columns[1:]
    tempmatrix = np.zeros((len(tweet_id),len(names)))   
    num_imagens = [0]*len(tweet_id)
    for i in media.index:
        index = np.where(tweet_id == media["tweet_id"].iloc[i])[0][0]
        j = num_imagens[index]
        tempmatrix[index,] += media.iloc[i,1:]
        num_imagens[index] += 1
        
    for i in np.arange(tweet_id.shape[0]):
        if (num_imagens[i]>1):
            tempmatrix[i,:] /= num_imagens[i]

    newmedia = pd.DataFrame(tempmatrix, index=range(tempmatrix.shape[0]), columns=names)
    newmedia["tweet_id"] = tweet_id

    return newmedia

#### 1.2.2 User

Combina os dados dos usuários antes de juntar com os dados dos tweets.

In [ ]:
#Junta as tabelas
users = users.merge(user_vectorized_descriptions, on = "user_id", how = "left")
users = users.merge(user_vectorized_profile_images, on = "user_id", how = "left")

#### 1.2.3 Combinando dados

In [ ]:
#Juntando dados dos tweets
train_tweets = train_tweets.merge(train_tweets_vectorized_text, on = "tweet_id", how = "left")
test_tweets = test_tweets.merge(test_tweets_vectorized_text, on = "tweet_id", how = "left")

In [ ]:
train = pd.merge(train_tweets, users, how='left', left_on='tweet_user_id', right_on='user_id')
train.drop(["tweet_id", "user_id", "tweet_user_id"], axis = 1, inplace = True)

test = pd.merge(test_tweets, users, how='left', left_on='tweet_user_id', right_on='user_id')
testId = test.loc[:, "tweet_id"]
test.drop(["tweet_id", "user_id", "tweet_user_id"], axis = 1, inplace = True)

Como temos que a variável `tweet_topic_ids` é categórica com muitas classes e pretendemos transforma-la em Dummies, optamos por excluí-la da análise, pois ela retornará muitas colunas com valores 0.

In [ ]:
#Remove a coluna tweet_topic_ids, pq tava gastando muito tempo pra tentar limpar ela
# e já conseguimos um resultado muito satisfatório sem ela, como será visto na sequência.
train.drop("tweet_topic_ids", axis = 1, inplace = True)
test.drop("tweet_topic_ids", axis = 1, inplace = True)

Converte a única variável categórica do nosso conjunto em numérica, usando o comando `get_dummies`.

In [ ]:
#Dummy 
train = pd.concat([train, pd.get_dummies(train.tweet_attachment_class)], axis = 1) 
train.drop("tweet_attachment_class", axis = 1, inplace = True)

test = pd.concat([test, pd.get_dummies(test.tweet_attachment_class)], axis = 1) 
test.drop("tweet_attachment_class", axis = 1, inplace = True)

In [ ]:
for col_name in train.columns: 
    print(col_name)

### 1.3 Boxplot de alguns atributos em função da viralidade

#### 1.3.1 user_like_count

In [ ]:
sns.boxplot(y='user_like_count', x='virality', 
                 data=train, 
                 palette="colorblind")

#### 1.3.2 tweet_hashtag_count

In [ ]:
sns.boxplot(y='tweet_hashtag_count', x='virality', 
                 data=train, 
                 palette="colorblind")

#### 1.3.3 user_followers_count

In [ ]:
sns.boxplot(y='user_followers_count', x='virality', 
                 data=train, 
                 palette="colorblind")

#### 1.3.4 user_tweet_count

In [ ]:
sns.boxplot(y='user_tweet_count', x='virality', 
                 data=train, 
                 palette="colorblind")

In [ ]:
print(train.shape)
print(test.shape)

## 2. Redução de dimensionalidade e treinamento de modelos.

Dado a dimensão dos dados ser muito grande, vamos utilizar alguns métodos diferentes visando reduzir a dimensionalidade. Após reduzirmos  a dimensionalidade, vamos trainar e validar os modelos, para ver qual método produz um resultado melhor.


### 2.1 PCA

Pela grande dimensionalidade do conjunto (3606 atributos), decidimos aplicar PCA em todos os atributos do nosso conjunto, como uma forma de reduzir a dimensionalidade para podermos utilizar em modelos e fazer as análises que queremos.

In [ ]:
columns = train.columns.values

y_columns = ["virality"]
x_columns = [x for x in columns if x != "virality"]

X = train[x_columns]
y = train[y_columns]

#PCA
pca = PCA(n_components = 500)

pca.fit(X)

X_PCA = pca.transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 2, stratify = y)
X_train_PCA, X_test_PCA, y_train_PCA, y_test_PCA = train_test_split(X_PCA, y, test_size = 0.2, random_state = 2, stratify = y)

Embora a variância seja explicada apenas pelos primeiros dez componentes, decidimos manter os 500 primeiros, pois o nosso receio era perder informação importante para a classificação que não foi captada pelos primeiros componentes. Como os atributos são muito diversos, incluindo desde informação dos tweets (texto), como informação do usuário (número de seguidores, número de likes, imagem do profile, entre outros), usando apenas os primeiros componentes limitaria o tipo de informação contida neles. Isso poderia deixar de fora informação importante, cuja variância é baixa. Verificamos essa hipótese testando os modelos com o número de componentes entre 10 e 500. Os melhores resultados no treinamento foi com 500 componentes.

In [ ]:
#Plot variância explicada acumulada
plt.plot(np.cumsum(pca.explained_variance_ratio_[0:10]))
plt.xlabel('Número de componentes')
plt.ylabel('Variância explicada acumulada');

Como podemos ver pelo gráfico acima, temos que apenas com 2 componentes principais já conseguimos explicar quase 100% da variabilidade dos nossos dados.

#### 2.1.1 Treinando e validando modelos com PCA

Ajuste dos hiperparâmetros do modelo XGBoost (comentado devido à demora em executá-lo).

In [ ]:
# # hiperparametros para ajustar
# param_grid = {
#         'min_child_weight': [3, 9, 18],
#         'gamma': [1, 3, 6, 9],
#         'subsample': [0.6, 0.8, 1.0],
#         'colsample_bytree': [0.6, 0.8, 1.0],
#         'max_depth': [3, 4, 5],
#         'n_estimators': [90,100,180],
#         }

# param_grid = {
#         'n_estimators': [180],
#         }

# gs_ab = GridSearchCV(XGBClassifier(), param_grid = param_grid, n_jobs=8, scoring = "accuracy")

# gs_ab.fit(X_train_PCA,y)

# print('Melhores parâmetros:', gs_ab.best_params_)

In [ ]:
# #model = XGBClassifier(colsample_bytree = gs_ab.best_params_['colsample_bytree'], gamma = gs_ab.best_params_['gamma'],
# #                      max_depth = gs_ab.best_params_['max_depth'], min_child_weight= gs_ab.best_params_['min_child_weight'],  subsample = gs_ab.best_params_['subsample'] )

model = XGBClassifier(n_estimators = 180)

model.fit(X_train, y_train.to_numpy().ravel())

y_pred = model.predict(X_test)
print("Acc. Score: {}".format(accuracy_score(y_test, y_pred)))

### 2.2 LDA

Ao utilizarmos o LDA, temos uma abordagem muito semelhante ao PCA, onde reduzimos a dimensionalidade por meio de uma transformação linear, porém, considerando as classes.

In [ ]:
lda = LinearDiscriminantAnalysis(n_components=4)

lda.fit(X_train,y_train)
y_pred = lda.predict(X_test)
print(lda.score(X_test,y_test))

### 2.3 Univariate Feature Selection

Ainda levando em conta a grande dimensionalidade do conjunto, decidimos testar outra abordagem, que seria a de seleção de features univariada (dado que a recursiva nesse nosso caso seria inviável devido a quantidade de atributos). Primeiramente testamos a seleção de feature com a função score f_classif





In [ ]:
X_train.shape

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import mutual_info_classif 

test = SelectKBest(score_func = f_classif, k=10)
fit = test.fit(X_train, y_train)

mask = fit.get_support() #list of booleans for selected features
new_feat = [] 
for bool, feature in zip(mask, X_train.columns):
    if bool:
        new_feat.append(feature)
print('The best features are:{}'.format(new_feat))

Segue abaixo os dados com as features que o método retornou

In [ ]:
X_train1 = X_train.loc[:,new_feat]
X_test1 = X_test.loc[:,new_feat]

Também utilizamos a função mutual_info_classif

In [ ]:
test = SelectKBest(score_func = mutual_info_classif, k=10)
fit = test.fit(X_train, y_train)

mask = fit.get_support() #list of booleans for selected features
new_feat = [] 
for bool, feature in zip(mask, X_train.columns):
    if bool:
        new_feat.append(feature)
print('The best features are:{}'.format(new_feat))

Segue abaixo os dados com as features que o método retornou

In [ ]:
X_train2 = X_train.loc[:,new_feat]
X_test2 = X_test.loc[:,new_feat]

#### 2.3.1 Treinando e validando modelos com Feature Selection Univariada

Avaliando acurácia com os dados obtidos da feature selection com função 
f_classif

In [ ]:
model = XGBClassifier(n_estimators = 180)
model.fit(X_train1, y_train.to_numpy().ravel())
y_pred1 = model.predict(X_test1)
print("Acc. Score: {}".format(accuracy_score(y_test, y_pred1)))

Avaliando acurácia com os dados obtidos da feature selection com função 
mutual_info_classif

In [ ]:
model = XGBClassifier(n_estimators = 180)
model.fit(X_train2, y_train.to_numpy().ravel())
y_pred2 = model.predict(X_test2)
print("Acc. Score: {}".format(accuracy_score(y_test, y_pred2)))

Como podemos ver, a acurácia obtida utilizando a seleção de features (com ambas as funções) e o LDA se saiu inferior ao resultado de quando utilizamos o PCA, portanto, vamos seguir com a abordagem utilizando PCA para classificarmos no conjunto de teste.

## 3. Resultado público do conjunto de teste

![alt text](df2cef56-2b05-4079-a724-ac331740e0cd.jpg "Resultado do bitgrit")

## 4. Conclusão

Neste trabalho, desenvolvemos algumas estratégias para a classificação do nível de viralidade dos tweets. A maior dificuldade deste trabalho foi lidar com um número elevado de atributos, muito dos quais não tinham uma relação clara com o nível de viralidade dos tweets. Assim, testamos dois métodos de redução de dimensionalidade: PCA e LDA. O primeiro transforma as variáveis preditoras em componentes com as mairores variâncias dos dados de forma descrescente. O segundo é similar ao primeiro, mas também leva em conta as classes dos dados para determinar os eixos de transformação. Também testamos dois métodos de seleção de atributos f_classif e mutual_info_classif para usar nos classificadores.

Devido ao volume enorme de dados do problema, optamos por testar apenas o modelo de classificação XGBoost no espaço reduzido de atributos. No caso da redução por LDA, também usamos esse método para classificar a viralidade dos tweets. Além disso, decidimos não usar os dados das mídias porque não conseguimos boa acurácia, usando apenas estes dados.

Com estas estratégias, conseguimos uma acurácia de quase 67\% no conjunto de teste separado do conjunto de treinamento usando o modelo XGBoost com PCA. O modelo LDA também apresentou um bom resultado, ficando em 66\%. Os resultados do XGBoost com a seleção de atributos ficou em torno de 64\%. Na submissão do `bitgrit`, conseguimos uma acurácia de 67.5\% usando o modelo XGBoost com PCA de 500 componentes.

Foi interessante participar de uma competição real, porém não tivemos tempo suficiente para explorar melhor outras estratégias de engenharia de atributos nem outros modelos de redução de atributos que levam em conta a variável preditora, como o próprio LDA. Além disso, como a base de dados era muito grande, muitos dos algoritmos que estávamos testando levaram horas para executar, limitando assim nossa capacidade de testar outras estratégias.